In [4]:
import pandas as pd
import numpy
import sklearn
import bioacoustics_model_zoo as bmz

In [5]:
train_df = pd.read_csv(
    "data/train_labels_5s_mac_frommeta.csv",
    index_col=[0,1,2]
)

val_df = pd.read_csv(
    "data/val_labels_5s_mac.csv",
    index_col=[0,1,2]
)

test_df = pd.read_csv(
    "data/test_labels_5s_mac_frommeta.csv",
    index_col=[0,1,2]
)

In [6]:

m = bmz.BirdSetEfficientNetB1()

X_val = m.embed(
    val_df,
    batch_size=32,
    num_workers=0
)

print(X_val.shape)
X_val.to_csv("data/Xval_birdset_effnetB1_5s.csv")


Loading weights:   0%|          | 0/508 [00:00<?, ?it/s]

/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

                    The function can also set the returned object's .constructor_name to the registered string key in ARCH_DICT
                    to avoid this warning and ensure it is reloaded correctly by opensoundscape.ml.load_model().

                    See opensoundscape.ml.cnn_architectures module for examples of constructor functions
                    
  warnings.warn(
/U

  0%|          | 0/47 [00:00<?, ?it/s]

/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  super().__init__(loader)


(1476, 1280)


In [7]:
import pandas as pd
import bioacoustics_model_zoo as bmz

train_df = pd.read_csv("data/train_labels_5s_mac_frommeta.csv", index_col=[0,1,2])
test_df  = pd.read_csv("data/test_labels_5s_mac_frommeta.csv",  index_col=[0,1,2])

m = bmz.BirdSetEfficientNetB1()

X_train = m.embed(train_df, batch_size=32, num_workers=0)
X_test  = m.embed(test_df,  batch_size=32, num_workers=0)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

X_train.to_csv("data/Xtrain_birdset_effnetB1_5s.csv")
X_test.to_csv("data/Xtest_birdset_effnetB1_5s.csv")


Loading weights:   0%|          | 0/508 [00:00<?, ?it/s]

/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

                    The function can also set the returned object's .constructor_name to the registered string key in ARCH_DICT
                    to avoid this warning and ensure it is reloaded correctly by opensoundscape.ml.load_model().

                    See opensoundscape.ml.cnn_architectures module for examples of constructor functions
                    
  warnings.warn(
/U

  0%|          | 0/319 [00:00<?, ?it/s]

/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  super().__init__(loader)


  0%|          | 0/47 [00:00<?, ?it/s]

/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  super().__init__(loader)


X_train: (10201, 1280)
X_test: (1476, 1280)


In [8]:
X_train = pd.read_csv("data/Xtrain_birdset_effnetB1_5s.csv", index_col=[0,1,2])
X_val   = pd.read_csv("data/Xval_birdset_effnetB1_5s.csv",   index_col=[0,1,2])
X_test  = pd.read_csv("data/Xtest_birdset_effnetB1_5s.csv",  index_col=[0,1,2])

y_train = pd.read_csv("data/train_labels_5s_mac_frommeta.csv", index_col=[0,1,2])
y_val   = pd.read_csv("data/val_labels_5s_mac.csv",            index_col=[0,1,2])
y_test  = pd.read_csv("data/test_labels_5s_mac_frommeta.csv",  index_col=[0,1,2])

# align to be safe (prevents subtle ordering bugs)
X_train = X_train.loc[y_train.index]
X_val   = X_val.loc[y_val.index]
X_test  = X_test.loc[y_test.index]

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(10201, 1280) (10201, 19)
(1476, 1280) (1476, 19)
(1476, 1280) (1476, 19)


In [11]:
import numpy as np
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score
import joblib

# 1) Train a shallow classifier (fast, strong baseline)
clf = OneVsRestClassifier(
    LogisticRegression(max_iter=3000, class_weight="balanced", n_jobs=-1),
    n_jobs=-1
)

clf.fit(
    X_train.values.astype(np.float32),
    y_train.values.astype(int)
)

# 2) Predict probabilities
p_val  = clf.predict_proba(X_val.values.astype(np.float32))
p_test = clf.predict_proba(X_test.values.astype(np.float32))

# 3) Evaluate macro mAP + per-class AP (skip classes with 0 positives)
label_names = y_train.columns.tolist()

def eval_split(y_df, p, name):
    y = y_df.values.astype(int)
    macro_map = average_precision_score(y, p, average="macro")
    print(f"\n{name} macro mAP: {macro_map:.4f}")

    per_class = {}
    for i, cls in enumerate(label_names):
        if y[:, i].sum() == 0:
            per_class[cls] = np.nan
        else:
            per_class[cls] = average_precision_score(y[:, i], p[:, i])

    s = pd.Series(per_class).sort_values()
    print("\nWorst AP (defined):")
    print(s.dropna().head(8))
    print("\nBest AP:")
    print(s.dropna().tail(8))
    missing = s[s.isna()].index.tolist()
    if missing:
        print("\nMissing in this split (0 positives):")
        print(missing)

eval_split(y_val, p_val, "VAL")
eval_split(y_test, p_test, "TEST")

# 4) Save the trained classifier
joblib.dump(clf, "data/shallow_lr_birdset_effnetB1.joblib")
print("\nSaved: data/shallow_lr_birdset_effnetB1.joblib")


/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/utils/parallel.py:135: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/utils/parallel.py:135: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/utils/parallel.py:135: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib worker


VAL macro mAP: 0.5057

Worst AP (defined):
Philepitta_castanea           0.007267
Treron_australis              0.076923
Eulemur_sp                    0.085460
Cyanolanius_madagascarinus    0.435897
Ceblepyris_cinereus           0.521277
Coua_caerulea                 0.584945
Leptopterus_chabert           0.617572
Copsychus_albospecularis      0.678102
dtype: float64

Best AP:
Neodrepanis_coruscans          0.689922
Agapornis_canus                0.709827
Eurystomus_glaucurus           0.833547
Coracopsis_nigra               0.844300
Hypsipetes_madagascariensis    0.860719
Zosterops_maderaspatanus       0.867953
Dicrurus_forficatus            0.871519
Saxicola_torquatus             0.922558
dtype: float64

Missing in this split (0 positives):
['Nesoenas_picturatus', 'Coua_reynaudii', 'Coua_cristata']

TEST macro mAP: 0.5203

Worst AP (defined):
Eulemur_sp                    0.064329
Leptopterus_chabert           0.093781
Agapornis_canus               0.180169
Neodrepanis_coruscans    

/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/sholmes3/miniforge3/envs/rewilding_pytorch_mac/lib/python3.10/site-packages/sklearn/metrics/_ranking.